In [174]:
import numpy as np
import gym
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import RMSprop
from keras import optimizers

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy, BoltzmannQPolicy, GreedyQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy, SoftmaxPolicy
from rl.core import Processor

In [175]:
def create_agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [176]:
class TaxiProcessor(Processor):
    def process_observation(self, observation):
        one_hot = np.zeros(500)
        one_hot[observation] = 1
        return one_hot

    def process_reward(self, reward):
        return reward

In [177]:
env = gym.make("Taxi-v3")
print("No of state = ", env.observation_space.n)
print("No of actions =", env.action_space.n)
model = create_agent(500, env.action_space.n)
model.summary()

No of state =  500
No of actions = 6
Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_26 (Flatten)        (None, 500)               0         
                                                                 
 dense_104 (Dense)           (None, 24)                12024     
                                                                 
 dense_105 (Dense)           (None, 24)                600       
                                                                 
 dense_106 (Dense)           (None, 24)                600       
                                                                 
 dense_107 (Dense)           (None, 6)                 150       
                                                                 
Total params: 13,374
Trainable params: 13,374
Non-trainable params: 0
_________________________________________________________________


In [178]:
memory = SequentialMemory(limit=800000, window_length=1)
processor = TaxiProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)

#policy = LinearAnnealedPolicy(MaxBoltzmannQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, processor=processor, memory=memory, nb_steps_warmup=1000, gamma=0.99, policy=policy, enable_double_dqn=False, target_model_update=5e-4)



In [179]:
optimizer = tf.keras.optimizers.legacy.RMSprop(lr=1e-3)
optimizer1 = optimizers.Adam(lr=1e-3)
dqn.compile(optimizer1, metrics=['mae'])
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 105s 10ms/step - reward: -2.4796
51 episodes - episode_reward: -485.745 [-821.000, -170.000] - loss: 0.249 - mae: 3.888 - mean_q: -1.638 - mean_eps: 0.478 - prob: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: -1.1527
50 episodes - episode_reward: -227.200 [-281.000, -66.000] - loss: 0.116 - mae: 6.776 - mean_q: -4.875 - mean_eps: 0.050 - prob: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 111s 11ms/step - reward: -1.1221
53 episodes - episode_reward: -211.264 [-263.000, 11.000] - loss: 0.188 - mae: 9.193 - mean_q: -7.275 - mean_eps: 0.050 - prob: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: -1.1314
62 episodes - episode_reward: -185.774 [-281.000, 11.000] - loss: 0.315 - mae: 11.058 - mean_q: -8.642 - m

In [180]:
env.reset()
res = dqn.test(env, nb_episodes=100, visualize=False)


Testing for 100 episodes ...
Episode 1: reward: 10.000, steps: 11
Episode 2: reward: 9.000, steps: 12
Episode 3: reward: 8.000, steps: 13
Episode 4: reward: 8.000, steps: 13
Episode 5: reward: 3.000, steps: 18
Episode 6: reward: 12.000, steps: 9
Episode 7: reward: 7.000, steps: 14
Episode 8: reward: 9.000, steps: 12
Episode 9: reward: 7.000, steps: 14
Episode 10: reward: 8.000, steps: 13
Episode 11: reward: 6.000, steps: 15
Episode 12: reward: 6.000, steps: 15
Episode 13: reward: 5.000, steps: 16
Episode 14: reward: 5.000, steps: 16
Episode 15: reward: 12.000, steps: 9
Episode 16: reward: 11.000, steps: 10
Episode 17: reward: 8.000, steps: 13
Episode 18: reward: 9.000, steps: 12
Episode 19: reward: 4.000, steps: 17
Episode 20: reward: 6.000, steps: 15
Episode 21: reward: 5.000, steps: 16
Episode 22: reward: 11.000, steps: 10
Episode 23: reward: 7.000, steps: 14
Episode 24: reward: 9.000, steps: 12
Episode 25: reward: 8.000, steps: 13
Episode 26: reward: 3.000, steps: 18
Episode 27: rew

In [181]:
res.history
print(np.average(res.history['episode_reward']))

5.67
